# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d47addd645-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Rome explorative search") 

Consider the following exploratory information need:

> The goal of this workflow is to study the Ancient Rome and some of the main events connected to that historical period. For instance, the emperors, killings, wars and so on.

## Useful URIs for the current workflow
The following data are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1048`| Julius Caesar       | node |





Also consider that

```
wd:Q1048 ?p ?obj .
```

is the BGP to retrieve all **properties of Julius Caesar**.

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Ancient Rome

2. Find all the Roman Dictators (return the IRI and the name of the dictator)

3. Identify the BGP for Roman Empire and Roman Republic

4. Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

5. Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

6. Find the BGP for war

7. Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

8. Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).


## Task 1
Identify the BGP for Ancient Rome

To start, I've run the example query "as is" to check everything is working fine.

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1048 ?p ?obj . #all properties of Julius Caesar
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1035'), ('name', 'honorific suffix')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/p

To check if I was understanding everything the right way, I tried to write a random query on my own using the results of the example query above.
This should return the language (wdt:P103) spoken by Julius Caesar.


In [3]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P103 ?obj . #language spoken by Julius Caesar
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q397'), ('name', 'Latin')]
1


Now it's time to work for the actual task. I realize the example query has been limited, so I run it again with LIMIT 100.

In [4]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1048 ?p ?obj . #all properties of Julius Caesar
   # get the label
   ?p sc:name ?name.
}
LIMIT 100 
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1035'), ('name', 'honorific suffix')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/p

They are a ton. Should I filter them using a Regex? Actually, I realize it may be way more trivial than that. I do not need to check the properties of Julius Caesar, I've been given the Nationality identifier in the table at the beginning. 

Final query for this task

In [5]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P27 ?obj . #nationality of Julius Caesar
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1747689'), ('name', 'Ancient Rome')]
1


## Task 2
Find all the Roman Dictators (return the IRI and the name of the dictator)

I had a lot of trouble for this one, cause I insisted to much about treating dictator as an occupation.

In [6]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P106 ?obj . #get all occupations of Julius Caesar
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3409374'), ('name', 'Ancient Roman priest')]
[('obj', 'http://www.wikidata.org/entity/Q99899510'), ('name', 'Ancient Roman historian')]
[('obj', 'http://www.wikidata.org/entity/Q1097498'), ('name', 'ruler')]
[('obj', 'http://www.wikidata.org/entity/Q12859263'), ('name', 'orator')]
[('obj', 'http://www.wikidata.org/entity/Q1402561'), ('name', 'military leader')]
[('obj', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('obj', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('obj', 'http://www.wikidata.org/entity/Q97667506'), ('name', 'Ancient Roman politician')]
[('obj', 'http://www.wikidata.org/entity/Q98103687'), ('name', 'Ancient Roman military personnel')]
[('obj', 'http://www.wikidata.org/entity/Q11774156'), ('name', 'memoirist')]
10


Ruler looked like a good starting point:

In [7]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P106 ?obj . #get all occupations of Julius Caesar
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3409374'), ('name', 'Ancient Roman priest')]
[('obj', 'http://www.wikidata.org/entity/Q99899510'), ('name', 'Ancient Roman historian')]
[('obj', 'http://www.wikidata.org/entity/Q1097498'), ('name', 'ruler')]
[('obj', 'http://www.wikidata.org/entity/Q12859263'), ('name', 'orator')]
[('obj', 'http://www.wikidata.org/entity/Q1402561'), ('name', 'military leader')]
[('obj', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('obj', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('obj', 'http://www.wikidata.org/entity/Q97667506'), ('name', 'Ancient Roman politician')]
[('obj', 'http://www.wikidata.org/entity/Q98103687'), ('name', 'Ancient Roman military personnel')]
[('obj', 'http://www.wikidata.org/entity/Q11774156'), ('name', 'memoirist')]
10


Actually, in the query above I did not realize I renamed just half vars, and thus the result is meaningless. Actually, this is a common mistake (you'll see I did it later on multiple times).

Let's retry with the right query:

In [8]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   # bind something
   ?person wdt:P106 wd:Q1097498 ; # people who have occupation = ruler
           wdt:P27  wd:Q1747689 . # and nationality = ancient rome
   # get the label
   ?person sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1291385'), ('name', 'Maria')]
[('person', 'http://www.wikidata.org/entity/Q2604067'), ('name', 'Flavia Titiana')]
[('person', 'http://www.wikidata.org/entity/Q1048'), ('name', 'Julius Caesar')]
[('person', 'http://www.wikidata.org/entity/Q191956'), ('name', 'Majorian')]
[('person', 'http://www.wikidata.org/entity/Q232094'), ('name', 'Flavia Maximiana Theodora')]
[('person', 'http://www.wikidata.org/entity/Q45530'), ('name', 'Eutropia')]
[('person', 'http://www.wikidata.org/entity/Q2479052'), ('name', 'Manlia Scantilla')]
[('person', 'http://www.wikidata.org/entity/Q239010'), ('name', 'Domitia Longina')]
[('person', 'http://www.wikidata.org/entity/Q241466'), ('name', 'Annia Faustina')]
[('person', 'http://www.wikidata.org/entity/Q1434'), ('name', 'Commodus')]
[('person', 'http://www.wikidata.org/entity/Q232271'), ('name', 'Justa Grata Honoria')]
[('person', 'http://www.wikidata.org/entity/Q46837'), ('name', 'Hostilian')]
[('person', '

There is something wrong with this query. Never heard about a female dictator as Flavia Titiana.

Maybe ruler is too broad, or maybe there is something else. In the next queries I tried to investigate on this with little results.

In [9]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2604067 wdt:P106 ?obj . #get all occupations of Flavia Titiana
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1097498'), ('name', 'ruler')]
1


In [10]:

queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   # bind something
   wd:Q1097498 ?p ?obj . #all properties of Ruler
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q12737077'), ('objName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q372436'), ('objName', 'statesperson')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q4086271'), ('objName', 'Bible Encyclopedia of Archimandrite Nicephorus')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1251441'), ('objName', 'leader')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of'), ('obj', 'http://www.wikidata.org/entity/Q1204348'), ('objName', 'subject')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15296531'), ('objName', 'position of authority')]
[('p'

In [11]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1048 ?p wd:Q1747689 . # Julius Caesar ?? Rome
   # get the label
   ?p sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
1


In [12]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1747689 ?p wd:Q1048 . # Rome ?? Julius Caesar
   # get the label
   ?p sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [13]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?p wdt:P279 wd:Q15296531 . # subclass of person of power
   # get the label
   ?p sc:name ?name.
}
LIMIT 100 
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1136723'), ('name', 'usurper')]
[('p', 'http://www.wikidata.org/entity/Q1097498'), ('name', 'ruler')]
2


Since I was stucked with no hope, I completely changed my approach. Maybe Julius Caesar is too complex to start with. Maybe I may look for someone is renewed for being a dictator. I immediately recall the most famous roman dictator ever: Sulla. 

However, finding Sulla on wikidata proved to be extremely difficult for four main reasons:
- the dictator may be spelled either Sulla or Silla, and I was not sure what was the right one
- for some weird reason, the entry about Sulla in wikidata is missing his full name
- there are tons of people with the same name in the Roman Empire
- the istance of the database I'm working on is missing labels and disambiguation labels, and there is no way I could see them unless I cheat. I would strongly advise to add them to the database next year, because they would have been really useful.

In [14]:
queryString = """
SELECT DISTINCT ?prenomen ?gensnomen ?cognomen
WHERE {
   # bind something
   wd:Q1048 wdt:P2358  ?prenomen ; # Name and 
            wdt:P2359  ?gensnomen; # gens name and
            wdt:P2365  ?cognomen . # surname of Julius Caesar.
           
   # Not sure if I have to get the label
   #?person sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('prenomen', 'http://www.wikidata.org/entity/Q5517401'), ('gensnomen', 'http://www.wikidata.org/entity/Q29871072'), ('cognomen', 'http://www.wikidata.org/entity/Q1025414')]
1


In [15]:
queryString = """
SELECT DISTINCT ?prenomen ?gensnomen ?cognomen
WHERE {
   # bind something
   wd:Q1048 wdt:P2358  ?prenomenr ; # Name and 
            wdt:P2359  ?gensnomenr; # gens name and
            wdt:P2365  ?cognomenr . # surname of Julius Caesar.
           
   # Ok, I have to get the label anyway
   ?prenomenr sc:name ?prenomen.
   ?gensnomenr sc:name ?gensnomen.
   ?cognomenr sc:name ?cognomen.
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('prenomen', 'Gaius'), ('gensnomen', 'Julius'), ('cognomen', 'Caesar')]
1


In [16]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   # bind something
   ?person wdt:P106 wd:Q1097498 ; # people who have nationality = ancient rome, then get:
           wdt:P2358  ?prenomenr ; # name and 
           wdt:P2359  ?gensnomenr; # gens name and
           wdt:P2365  ?cognomenr . # surname of those people.
           
   # get the labels and use them to filter
   ?prenomenr sc:name ?prenomen.
   ?gensnomenr sc:name ?gensnomen.
   ?cognomenr sc:name ?cognomen.
   
   FILTER regex(?prenomen, "Lucius")
   FILTER regex(?gensnomen, "Cornelius")
   FILTER regex(?cognomen, "Sulla")

   
   
   # get the label of the result
   ?person sc:name ?name.
   
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [17]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   # bind something
   ?person wdt:P106 wd:Q1097498 ; # people who have nationality = ancient rome, then get:
           wdt:P2358  ?prenomenr ; # name and 
           wdt:P2359  ?gensnomenr; # gens name and
           wdt:P2365  ?cognomenr . # surname of those people.
           
   # get the labels and use them to filter
   ?prenomenr sc:name ?prenomen.
   ?gensnomenr sc:name ?gensnomen.
   ?cognomenr sc:name ?cognomen.
   
   FILTER regex(?prenomen, "Gaius")
   FILTER regex(?gensnomen, "Julius")
   FILTER regex(?cognomen, "Caesar")

   
   
   # get the label of the result
   ?person sc:name ?name.
   
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1048'), ('name', 'Julius Caesar')]
1


In [18]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   # bind something
   ?person wdt:P106 wd:Q1097498. # people who have nationality = ancient rome, then get:

   
   
   # get the label of the result
   ?person sc:name ?name.
    FILTER regex(?name, "Silla")
  
   
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q488377'), ('name', 'Jindeok of Silla')]
[('person', 'http://www.wikidata.org/entity/Q496772'), ('name', 'Hyegong of Silla')]
[('person', 'http://www.wikidata.org/entity/Q484898'), ('name', 'Hyeokgeose of Silla')]
3


In [19]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   # bind something
   ?person wdt:P106 wd:Q1097498. # people who have nationality = ancient rome, then get:

   
   
   # get the label of the result
   ?person sc:name ?name.
    FILTER regex(?name, "Lucius")
  
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [20]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person wdt:P31 wdt:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [21]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person ?smt wdt:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [22]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person ?smt wd:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto')]
[('person', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek')]
[('person', 'http://www.wikidata.org/entity/Q102115260'), ('name', 'Kojiro Kobayashi')]
[('person', 'http://www.wikidata.org/entity/Q102118153'), ('name', 'Miguel E. M. Herrera')]
[('person', 'http://www.wikidata.org/entity/Q105753880'), ('name', 'Mateo Egea')]
[('person', 'http://www.wikidata.org/entity/Q107520918'), ('name', 'Jerónimo Latorre')]
[('person', 'http://www.wikidata.org/entity/Q11404285'), ('name', 'Mikiko Kitano')]
[('person', 'http://www.wikidata.org/entity/Q11520274'), ('name', 'Honjō Fusanaga')]
[('person', 'http://www.wikidata.org/entity/Q11562694'), ('name', 'Takayoshi Watanabe')]
[('person', 'http://www.wikidata.org/entity/Q11685026'), ('name', 'Joan de Gènova')]
[('person', 'http://www.wikidata.org/entity/Q11686949'), ('name', 'José Ximénez')]
[('person', 'http://www.wikidata.org/entity/Q1

In [23]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person ?smt wd:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
   FILTER regex(?name, "Lucius")

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q16188292'), ('name', 'Lucius Caecilius')]
[('person', 'http://www.wikidata.org/entity/Q20010809'), ('name', 'Lucius Fenestella')]
[('person', 'http://www.wikidata.org/entity/Q22038936'), ('name', 'Lucius Cary, 6th Viscount Falkland')]
[('person', 'http://www.wikidata.org/entity/Q75270013'), ('name', 'Lucius Charles Cary, 7th Viscount of Falkland')]
[('person', 'http://www.wikidata.org/entity/Q26248409'), ('name', 'Lucius Rossi')]
[('person', 'http://www.wikidata.org/entity/Q120122'), ('name', 'Lucius Domitius Ahenobarbus')]
[('person', 'http://www.wikidata.org/entity/Q465212'), ('name', 'Lucius Domitius Ahenobarbus')]
[('person', 'http://www.wikidata.org/entity/Q507036'), ('name', 'Lucius Annaeus Cornutus')]
[('person', 'http://www.wikidata.org/entity/Q11934044'), ('name', 'Lucius Calpurnius Piso')]
[('person', 'http://www.wikidata.org/entity/Q1236713'), ('name', 'Lucius Marius Maximus')]
[('person', 'http://www.wikidata.org/entity/Q

In [24]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person ?smt wd:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
   FILTER regex(?name, "Lucius Cornelius")

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q4269984'), ('name', 'Lucius Cornelius Lentulus')]
[('person', 'http://www.wikidata.org/entity/Q201102'), ('name', 'Lucius Cornelius Balbus the Younger')]
[('person', 'http://www.wikidata.org/entity/Q958946'), ('name', 'Lucius Cornelius Lentulus')]
[('person', 'http://www.wikidata.org/entity/Q12129786'), ('name', 'Lucius Cornelius Lentulus')]
[('person', 'http://www.wikidata.org/entity/Q361377'), ('name', 'Lucius Cornelius Scipio Asiaticus')]
[('person', 'http://www.wikidata.org/entity/Q708483'), ('name', 'Lucius Cornelius Scipio')]
[('person', 'http://www.wikidata.org/entity/Q374630'), ('name', 'Lucius Cornelius Scipio Barbatus')]
[('person', 'http://www.wikidata.org/entity/Q1873583'), ('name', 'Lucius Cornelius Lentulus Lupus')]
[('person', 'http://www.wikidata.org/entity/Q20005189'), ('name', 'Lucius Cornelius Lentulus Caudinus')]
[('person', 'http://www.wikidata.org/entity/Q934699'), ('name', 'Lucius Cornelius Lentulus Caudinus')]

In [ ]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person ?smt wd:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
   FILTER regex(?name, "Lucius Cornelius Sulla")

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q329572'), ('name', 'Lucius Cornelius Sulla')]
[('person', 'http://www.wikidata.org/entity/Q716713'), ('name', 'Lucius Cornelius Sulla Felix')]
[('person', 'http://www.wikidata.org/entity/Q12119006'), ('name', 'Lucius Cornelius Sulla')]
3


In [ ]:
queryString = """
SELECT DISTINCT ?occupationr ?occupation
WHERE {
   wd:Q329572 wdt:P106 ?occupationr . # occupation of Silla
   # get the label
   ?occupationr sc:name ?occupation.
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('occupationr', 'http://www.wikidata.org/entity/Q47064'), ('occupation', 'military personnel')]
[('occupationr', 'http://www.wikidata.org/entity/Q82955'), ('occupation', 'politician')]
2


In [ ]:
queryString = """
SELECT DISTINCT ?occupationr ?occupation
WHERE {
   wd:Q12119006 wdt:P106 ?occupationr . # occupation of the other Silla
   # get the label
   ?occupationr sc:name ?occupation.
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
queryString = """
SELECT DISTINCT ?occupationr ?occupation
WHERE {
    wd:Q329572 ?is ?occupationr . # exploring Silla by regex
    # get the label
    ?occupationr sc:name ?occupation.
    FILTER regex(?occupation, "ictato")

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
queryString = """
SELECT DISTINCT ?occupationr ?occupation
WHERE {
    wd:Q329572 ?is ?occupationr . # exploring Silla without regex
    # get the label
    ?occupationr sc:name ?occupation.

}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('occupationr', 'http://www.wikidata.org/entity/Q11915639'), ('occupation', 'Cornelius')]
[('occupationr', 'http://www.wikidata.org/entity/Q221312'), ('occupation', 'Cornelia gens')]
[('occupationr', 'http://www.wikidata.org/entity/Q1365457'), ('occupation', 'Faustus Cornelius Sulla')]
[('occupationr', 'http://www.wikidata.org/entity/Q1747689'), ('occupation', 'Ancient Rome')]
[('occupationr', 'http://www.wikidata.org/entity/Q20056508'), ('occupation', 'Ancient Roman senator')]
[('occupationr', 'http://www.wikidata.org/entity/Q220'), ('occupation', 'Rome')]
[('occupationr', 'http://www.wikidata.org/entity/Q40779'), ('occupation', 'Roman consul')]
[('occupationr', 'http://www.wikidata.org/entity/Q47064'), ('occupation', 'military personnel')]
[('occupationr', 'http://www.wikidata.org/entity/Q5'), ('occupation', 'human')]
[('occupationr', 'http://www.wikidata.org/entity/Q6581097'), ('occupation', 'male')]
[('occupationr', 'http://www.wikidata.org/entity/Q6697451'), ('occupation'

In [ ]:
queryString = """
SELECT DISTINCT ?occupationr ?occupation
WHERE {
    wd:Q12119006 ?is ?occupationr . # exploring other Silla without regex
    # get the label
    ?occupationr sc:name ?occupation.

}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('occupationr', 'http://www.wikidata.org/entity/Q5'), ('occupation', 'human')]
[('occupationr', 'http://www.wikidata.org/entity/Q6581097'), ('occupation', 'male')]
[('occupationr', 'http://www.wikidata.org/entity/Q483783'), ('occupation', 'Sulla')]
[('occupationr', 'http://www.wikidata.org/entity/Q3621973'), ('occupation', 'Publius Cornelius Sulla')]
[('occupationr', 'http://www.wikidata.org/entity/Q15715169'), ('occupation', 'Cornelia')]
[('occupationr', 'http://www.wikidata.org/entity/Q12145490'), ('occupation', 'Publius Cornelius Sulla')]
6


In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q329572 ?pr ?or . # exploring Silla without regex
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'occupation'), ('or', 'http://www.wikidata.org/entity/Q47064'), ('o', 'military personnel')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'occupation'), ('or', 'http://www.wikidata.org/entity/Q82955'), ('o', 'politician')]
[('pr', 'http://www.wikidata.org/prop/direct/P19'), ('p', 'place of birth'), ('or', 'http://www.wikidata.org/entity/Q220'), ('o', 'Rome')]
[('pr', 'http://www.wikidata.org/prop/direct/P21'), ('p', 'sex or gender'), ('or', 'http://www.wikidata.org/entity/Q6581097'), ('o', 'male')]
[('pr', 'http://www.wikidata.org/prop/direct/P22'), ('p', 'father'), ('or', 'http://www.wikidata.org/entity/Q125545'), ('o', 'Publius Cornelius Sulla')]
[('pr', 'http://www.wikidata.org/prop/direct/P2348'), ('p', 'time period'), ('or', 'http://www.wikidata.org/entity/Q787204'), ('o', 'High Roman Empire')]
[('pr', 'http://www.wikidata.org/prop/direct/P2358'), ('p', 'Roman praenomen'), ('or', 'http://www.wikidata.

In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q329572 ?pr ?or . # exploring Sulla with no properties label

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('or', 'http://wikiba.se/ontology#Item')]
[('pr', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('or', 'http://g.co/kg/m/08z80q')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('or', 'http://www.wikidata.org/entity/Q47064')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('or', 'http://www.wikidata.org/entity/Q82955')]
[('pr', 'http://www.wikidata.org/prop/direct/P1317'), ('or', '-004-01-01T00:00:00Z')]
[('pr', 'http://www.wikidata.org/prop/direct/P1819'), ('or', 'I00735465')]
[('pr', 'http://www.wikidata.org/prop/direct/P19'), ('or', 'http://www.wikidata.org/entity/Q220')]
[('pr', 'http://www.wikidata.org/prop/direct/P21'), ('or', 'http://www.wikidata.org/entity/Q6581097')]
[('pr', 'http://www.wikidata.org/prop/direct/P22'), ('or', 'http://www.wikidata.org/entity/Q125545')]
[('pr', 'http://www.wikidata.org/prop/direct/P2348'), ('or', 'http://www.wikidata.org/entity/Q787204')]
[('pr', 'http://ww

In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q716713 ?pr ?or . # exploring the third Sulla
    # get the labels
    ?or sc:name ?o.
    ?pr sc:name ?p.

}
LIMIT 50 
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'occupation'), ('or', 'http://www.wikidata.org/entity/Q47064'), ('o', 'military personnel')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'occupation'), ('or', 'http://www.wikidata.org/entity/Q82955'), ('o', 'politician')]
[('pr', 'http://www.wikidata.org/prop/direct/P21'), ('p', 'sex or gender'), ('or', 'http://www.wikidata.org/entity/Q6581097'), ('o', 'male')]
[('pr', 'http://www.wikidata.org/prop/direct/P22'), ('p', 'father'), ('or', 'http://www.wikidata.org/entity/Q329572'), ('o', 'Lucius Cornelius Sulla')]
[('pr', 'http://www.wikidata.org/prop/direct/P2348'), ('p', 'time period'), ('or', 'http://www.wikidata.org/entity/Q787204'), ('o', 'High Roman Empire')]
[('pr', 'http://www.wikidata.org/prop/direct/P2358'), ('p', 'Roman praenomen'), ('or', 'http://www.wikidata.org/entity/Q6697451'), ('o', 'Lucius')]
[('pr', 'http://www.wikidata.org/prop/direct/P2359'), ('p', 'Roman nomen gentilicium'), ('or', 'http

In [ ]:
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   ?person ?smt wd:Q5. # peoples

   # get the label of the result
   ?person sc:name ?name.
    FILTER regex(?name, "Sulla")
  
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q238036'), ('name', 'Margaret Sullavan')]
[('person', 'http://www.wikidata.org/entity/Q1365457'), ('name', 'Faustus Cornelius Sulla')]
[('person', 'http://www.wikidata.org/entity/Q705117'), ('name', 'Faustus Cornelius Sulla Felix')]
[('person', 'http://www.wikidata.org/entity/Q353214'), ('name', 'Marcus Munatius Sulla Urbanus')]
[('person', 'http://www.wikidata.org/entity/Q678644'), ('name', 'Marcus Munatius Sulla Cerialis')]
[('person', 'http://www.wikidata.org/entity/Q11915650'), ('name', 'Cornelius Sulla')]
[('person', 'http://www.wikidata.org/entity/Q15709216'), ('name', 'Cornelia Sulla')]
[('person', 'http://www.wikidata.org/entity/Q483783'), ('name', 'Sulla')]
[('person', 'http://www.wikidata.org/entity/Q5438354'), ('name', 'Faustus Cornelius Sulla')]
[('person', 'http://www.wikidata.org/entity/Q708722'), ('name', 'Faustus Cornelius Sulla')]
[('person', 'http://www.wikidata.org/entity/Q1412864'), ('name', 'Julius Sullan Tonel')]

In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q483783 ?pr ?or . # exploring a new Sulla without regex
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P102'), ('p', 'member of political party'), ('or', 'http://www.wikidata.org/entity/Q309155'), ('o', 'optimates')]
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('p', 'relative'), ('or', 'http://www.wikidata.org/entity/Q380857'), ('o', 'Aemilia Scaura')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'occupation'), ('or', 'http://www.wikidata.org/entity/Q97667506'), ('o', 'Ancient Roman politician')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('p', 'occupation'), ('or', 'http://www.wikidata.org/entity/Q98103687'), ('o', 'Ancient Roman military personnel')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q1138524'), ('o', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q2657718'), ('o', 'Armeni

At the end, I finally realize the entities I was looking for were `wdt:P39` and `wd:Q236885`

Final query for this task

In [ ]:
queryString = """
SELECT DISTINCT ?dictatorr ?dictator
WHERE {
    ?dictatorr wdt:P39 wd:Q236885 . # All people with position held = roman dicator
    # get the label
    ?dictatorr sc:name ?dictator.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('dictatorr', 'http://www.wikidata.org/entity/Q155312'), ('dictator', 'Gaius Claudius Centho')]
[('dictatorr', 'http://www.wikidata.org/entity/Q297783'), ('dictator', 'Appius Claudius Caecus')]
[('dictatorr', 'http://www.wikidata.org/entity/Q703448'), ('dictator', 'Marcus Livius Salinator')]
[('dictatorr', 'http://www.wikidata.org/entity/Q529867'), ('dictator', 'Titus Lartius')]
[('dictatorr', 'http://www.wikidata.org/entity/Q15034789'), ('dictator', 'Manius Valerius Maximus')]
[('dictatorr', 'http://www.wikidata.org/entity/Q359810'), ('dictator', 'Lucius Caecilius Metellus')]
[('dictatorr', 'http://www.wikidata.org/entity/Q929498'), ('dictator', 'Quintus Caecilius Metellus')]
[('dictatorr', 'http://www.wikidata.org/entity/Q519186'), ('dictator', 'Quintus Hortensius')]
[('dictatorr', 'http://www.wikidata.org/entity/Q637596'), ('dictator', 'Quintus Servilius Priscus Structus Fidenas')]
[('dictatorr', 'http://www.wikidata.org/entity/Q1223522'), ('dictator', 'Caius Poetelius Libo

## Task 3
Identify the BGP for Roman Empire and Roman Republic

At first, I tried with some generic approaches:

In [ ]:
queryString = """
SELECT DISTINCT ?children ?name
WHERE {
   wd:Q1048 wdt:P22 ?children . # Children of Julius Caesar 

   # get the label of the result
   ?children sc:name ?name.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('children', 'http://www.wikidata.org/entity/Q314710'), ('name', 'Gaius Julius Caesar')]
1


In [ ]:
queryString = """
SELECT DISTINCT ?children ?name
WHERE {
    wd:Q1048 ?did ?thingsr . # Things done by Julius Caesar 

    # get the label of the result
    ?thingsr sc:name ?things.
    FILTER regex(?things, "empire", "i")
  
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


Then I realize I could just use the time period property discovered above.

In [ ]:
queryString = """
SELECT DISTINCT ?or ?o
WHERE {
    wd:Q1048 wdt:P2348 ?or . # Time period of Julius Caesar 

    # get the label of the result
    ?or sc:name ?o.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('or', 'http://www.wikidata.org/entity/Q2815472'), ('o', 'Late Roman Republic')]
1


In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q2815472 ?pr ?or . # Properties of the late republic 

    # get the label of the results
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('p', 'instance of'), ('or', 'http://www.wikidata.org/entity/Q11514315'), ('o', 'historical period')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('p', 'followed by'), ('or', 'http://www.wikidata.org/entity/Q787204'), ('o', 'High Roman Empire')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('p', 'part of'), ('or', 'http://www.wikidata.org/entity/Q17167'), ('o', 'Roman Republic')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('p', 'follows'), ('or', 'http://www.wikidata.org/entity/Q6106068'), ('o', 'Middle Roman Republic')]
4


In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q787204 ?pr ?or . # Properties of the high empire 

    # get the label of the results
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('p', 'follows'), ('or', 'http://www.wikidata.org/entity/Q2815472'), ('o', 'Late Roman Republic')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('p', 'instance of'), ('or', 'http://www.wikidata.org/entity/Q11514315'), ('o', 'historical period')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('p', 'part of'), ('or', 'http://www.wikidata.org/entity/Q2277'), ('o', 'Roman Empire')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('p', 'followed by'), ('or', 'http://www.wikidata.org/entity/Q2886278'), ('o', 'Later Roman Empire')]
4


Final query for this task

In [ ]:
queryString = """
SELECT DISTINCT ?period ?name
WHERE {
    VALUES ?specificperiod { wd:Q787204  wd:Q2815472 }
    ?specificperiod wdt:P361 ?period.  # specific period is part of period
    ?period sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('period', 'http://www.wikidata.org/entity/Q2277'), ('name', 'Roman Empire')]
[('period', 'http://www.wikidata.org/entity/Q17167'), ('name', 'Roman Republic')]
2


## Task 4
Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

First, I need to find the IRI for emperor. At first, I tried with some people that may be categorized as emperor (I tried with a couple of them, because I was not sure about who was an emperor):

In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q1048 ?pr ?or . # Julius Caesar objects (actually, I realized I've choosen the wrong example, he is not an emperor)
    FILTER regex(?o, "emperor", "i")
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q1048 ?pr ?or . # Julius Caesar predicates
    FILTER regex(?p, "emperor", "i")
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [ ]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q2604067 ?pr ?or . # checking another ruler from query 2
    FILTER regex(?p, "emperor", "i")
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


Then, I tried with regex, which proven to be more effective:

In [ ]:
queryString = """
SELECT DISTINCT ?personr ?person ?pr ?p ?or ?o
WHERE {
   ?personr wdt:P27  wd:Q1747689 ; # people who nationality = ancient rome
                 ?pr ?or.
                 
    FILTER regex(?o, "emperor", "i")
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.
    ?personr sc:name ?person.


}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('personr', 'http://www.wikidata.org/entity/Q159798'), ('person', 'Honorius'), ('pr', 'http://www.wikidata.org/prop/direct/P39'), ('p', 'position held'), ('or', 'http://www.wikidata.org/entity/Q842606'), ('o', 'Roman emperor')]
[('personr', 'http://www.wikidata.org/entity/Q46780'), ('person', 'Aurelian'), ('pr', 'http://www.wikidata.org/prop/direct/P39'), ('p', 'position held'), ('or', 'http://www.wikidata.org/entity/Q842606'), ('o', 'Roman emperor')]
[('personr', 'http://www.wikidata.org/entity/Q437604'), ('person', 'Julius Saturninus'), ('pr', 'http://www.wikidata.org/prop/direct/P39'), ('p', 'position held'), ('or', 'http://www.wikidata.org/entity/Q842606'), ('o', 'Roman emperor')]
[('personr', 'http://www.wikidata.org/entity/Q46780'), ('person', 'Aurelian'), ('pr', 'http://www.wikidata.org/prop/direct/P53'), ('p', 'family'), ('or', 'http://www.wikidata.org/entity/Q887928'), ('o', 'Illyrian emperors')]
[('personr', 'http://www.wikidata.org/entity/Q159798'), ('person', 'Hono

In [ ]:
queryString = """
SELECT DISTINCT ?or ?o
WHERE {
   ?personr wdt:P27  wd:Q1747689 ; # people who nationality = ancient rome
                 ?pr ?or.
                 
    FILTER regex(?o, "emperor", "i")
    # get the label
    ?or sc:name ?o.
    ?pr sc:name ?p.
    ?personr sc:name ?person.


}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('or', 'http://www.wikidata.org/entity/Q842606'), ('o', 'Roman emperor')]
[('or', 'http://www.wikidata.org/entity/Q887928'), ('o', 'Illyrian emperors')]
[('or', 'http://www.wikidata.org/entity/Q18577504'), ('o', 'Byzantine emperor')]
[('or', 'http://www.wikidata.org/entity/Q193254'), ('o', 'Five Good Emperors')]
[('or', 'http://www.wikidata.org/entity/Q39018'), ('o', 'Emperor')]
[('or', 'http://www.wikidata.org/entity/Q128866'), ('o', 'list of Byzantine emperors')]
[('or', 'http://www.wikidata.org/entity/Q3023913'), ('o', 'last emperors')]
[('or', 'http://www.wikidata.org/entity/Q9479093'), ('o', 'Category:Julian (emperor)')]
[('or', 'http://www.wikidata.org/entity/Q106602516'), ('o', "Familia Caesaris. A Social Study of the Emperor's Freedmen and Slaves")]
[('or', 'http://www.wikidata.org/entity/Q470596'), ('o', 'Year of the Four Emperors')]
10


Now I tried to get the `homicide` IRI (the first query has actually wrong vars name, the second is right):

In [70]:

queryString = """

"""

print("Results")
#x=run_query(queryString)

Results


In [ ]:
queryString = """
SELECT DISTINCT ?personr ?person ?deathr ?death
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

   ?personr wdt:P27  wd:Q1747689 ; # people who nationality = ancient rome
            ?occupation ?emperor;
            wdt:P1196 ?death .
            
    
            
    # get the label
    ?deathr sc:name ?death.
    ?personr sc:name ?person.


}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
Empty


I try to get the `killer` IRI, first with a regex:

*Note: actually, it was a good idea, however, there is a mistake on the regex, the filter should be on the `?killed` and not on `?killedr`*

In [ ]:
queryString = """
SELECT DISTINCT ?killerr ?killer ?killedr ?killed ?personr ?person
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

    ?personr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor;  # who are emperors
            wdt:P1196 wd:Q149086 . # and died by homicide
            
    # try to get a killer of that person throught regex
    ?killerr ?killedr ?personr.
    FILTER regex(?killedr, "kill", "i") 
            
    # get the label
    ?killerr sc:name ?killer.
    ?killedr sc:name ?killed.
    ?personr sc:name ?person.


}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
Empty


Since the regex failed, I tried to get all relationsh with killed people:

In [ ]:
queryString = """
SELECT DISTINCT ?killerr ?killer ?killedr ?killed ?personr ?person
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

    ?personr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor;  # who are emperors
            wdt:P1196 wd:Q149086 . # and died by homicide
            
    ?killerr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
           ?killedr ?personr.      # who did something to the killed person
            
            
    # get the label
    ?killerr sc:name ?killer.
    ?killedr sc:name ?killed.
    ?personr sc:name ?person.


}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('killerr', 'http://www.wikidata.org/entity/Q242199'), ('killer', 'Junia Claudilla'), ('killedr', 'http://www.wikidata.org/prop/direct/P26'), ('killed', 'spouse'), ('personr', 'http://www.wikidata.org/entity/Q1409'), ('person', 'Caligula')]
[('killerr', 'http://www.wikidata.org/entity/Q101062173'), ('killer', 'Flavia Domitiana'), ('killedr', 'http://www.wikidata.org/prop/direct/P22'), ('killed', 'father'), ('personr', 'http://www.wikidata.org/entity/Q1423'), ('person', 'Domitian')]
[('killerr', 'http://www.wikidata.org/entity/Q2803581'), ('killer', 'Titus Flavius Caesar'), ('killedr', 'http://www.wikidata.org/prop/direct/P22'), ('killed', 'father'), ('personr', 'http://www.wikidata.org/entity/Q1423'), ('person', 'Domitian')]
[('killerr', 'http://www.wikidata.org/entity/Q1777'), ('killer', 'Maximinus Thrax'), ('killedr', 'http://www.wikidata.org/prop/direct/P40'), ('killed', 'child'), ('personr', 'http://www.wikidata.org/entity/Q518890'), ('person', 'Gaius Julius Verus Maximus'

In [ ]:
queryString = """
SELECT DISTINCT ?killedr ?killed
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

    ?personr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor;  # who are emperors
            wdt:P1196 wd:Q149086 . # and died by homicide
            
    ?killerr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
           ?killedr ?personr.      # who did something to the killed person
            
            
    # get the label
    ?killerr sc:name ?killer.
    ?killedr sc:name ?killed.
    ?personr sc:name ?person.


}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('killedr', 'http://www.wikidata.org/prop/direct/P26'), ('killed', 'spouse')]
[('killedr', 'http://www.wikidata.org/prop/direct/P22'), ('killed', 'father')]
[('killedr', 'http://www.wikidata.org/prop/direct/P40'), ('killed', 'child')]
[('killedr', 'http://www.wikidata.org/prop/direct/P1038'), ('killed', 'relative')]
[('killedr', 'http://www.wikidata.org/prop/direct/P451'), ('killed', 'unmarried partner')]
[('killedr', 'http://www.wikidata.org/prop/direct/P1889'), ('killed', 'different from')]
[('killedr', 'http://www.wikidata.org/prop/direct/P3373'), ('killed', 'sibling')]
[('killedr', 'http://www.wikidata.org/prop/direct/P802'), ('killed', 'student')]
[('killedr', 'http://www.wikidata.org/prop/direct/P127'), ('killed', 'owned by')]
[('killedr', 'http://www.wikidata.org/prop/direct/P157'), ('killed', 'killed by')]
[('killedr', 'http://www.wikidata.org/prop/direct/P1327'), ('killed', 'partner in business or sport')]
11


Final query for this task (before feedback - after that I checked and realized I was switching the killer with the killed):

In [ ]:
queryString = """
SELECT DISTINCT ?personr ?person ?killerr ?killer
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

    ?personr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor;  # who are emperors
            wdt:P1196 wd:Q149086 . # and died by homicide
            
    OPTIONAL{
        ?killerr wdt:P157 ?personr. # people who did kill that person to the killed person
        
        # get the label
        ?killerr sc:name ?killer.
    }
            
            
    # get the label
    ?personr sc:name ?person.


}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('personr', 'http://www.wikidata.org/entity/Q1423'), ('person', 'Domitian'), ('killerr', 'http://www.wikidata.org/entity/Q21567913'), ('killer', 'Paris')]
[('personr', 'http://www.wikidata.org/entity/Q170026'), ('person', 'Valentinian III'), ('killerr', 'http://www.wikidata.org/entity/Q185476'), ('killer', 'Flavius Aetius')]
[('personr', 'http://www.wikidata.org/entity/Q1446'), ('person', 'Caracalla'), ('killerr', 'http://www.wikidata.org/entity/Q183089'), ('killer', 'Geta')]
[('personr', 'http://www.wikidata.org/entity/Q1434'), ('person', 'Commodus'), ('killerr', 'http://www.wikidata.org/entity/Q242466'), ('killer', 'Lucilla')]
[('personr', 'http://www.wikidata.org/entity/Q46780'), ('person', 'Aurelian')]
[('personr', 'http://www.wikidata.org/entity/Q437604'), ('person', 'Julius Saturninus')]
[('personr', 'http://www.wikidata.org/entity/Q1777'), ('person', 'Maximinus Thrax')]
[('personr', 'http://www.wikidata.org/entity/Q518890'), ('person', 'Gaius Julius Verus Maximus')]
[('

Final query for this task:

In [71]:
queryString = """
SELECT DISTINCT ?personr ?person ?killerr ?killer
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

    ?personr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor;  # who are emperors
            wdt:P1196 wd:Q149086 . # and died by homicide
            
    OPTIONAL{
        ?personr wdt:P157 ?killerr. # person killed by killer
        
        # get the label
        ?killerr sc:name ?killer.
    }
            
            
    # get the label
    ?personr sc:name ?person.


}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('personr', 'http://www.wikidata.org/entity/Q103860'), ('person', 'Julius Nepos'), ('killerr', 'http://www.wikidata.org/entity/Q1228819'), ('killer', 'Ovida')]
[('personr', 'http://www.wikidata.org/entity/Q189108'), ('person', 'Gratian'), ('killerr', 'http://www.wikidata.org/entity/Q2846204'), ('killer', 'Andragathius')]
[('personr', 'http://www.wikidata.org/entity/Q1434'), ('person', 'Commodus'), ('killerr', 'http://www.wikidata.org/entity/Q980693'), ('killer', 'Narcissus')]
[('personr', 'http://www.wikidata.org/entity/Q183089'), ('person', 'Geta'), ('killerr', 'http://www.wikidata.org/entity/Q1446'), ('killer', 'Caracalla')]
[('personr', 'http://www.wikidata.org/entity/Q46780'), ('person', 'Aurelian')]
[('personr', 'http://www.wikidata.org/entity/Q437604'), ('person', 'Julius Saturninus')]
[('personr', 'http://www.wikidata.org/entity/Q1777'), ('person', 'Maximinus Thrax')]
[('personr', 'http://www.wikidata.org/entity/Q518890'), ('person', 'Gaius Julius Verus Maximus')]
[('pe

## Task 5
Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

Final query for this task

In [ ]:
queryString = """
SELECT DISTINCT ?killerr ?killer ?killedr ?killed
WHERE {
    VALUES ?emperor { wd:Q842606  wd:Q39018 } # check both "Emperor" and "Roman emperor"

    ?killedr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor.  # who are emperors
            
    ?killerr wdt:P27  wd:Q1747689; # people who nationality = ancient rome
            ?occupation ?emperor;  # who are emperors
            wdt:P157 ?killedr.     # and killed another emperor

 
    # get the label
    ?killerr sc:name ?killer.
    ?killedr sc:name ?killed.


}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('killerr', 'http://www.wikidata.org/entity/Q183089'), ('killer', 'Geta'), ('killedr', 'http://www.wikidata.org/entity/Q1446'), ('killed', 'Caracalla')]
[('killerr', 'http://www.wikidata.org/entity/Q184549'), ('killer', 'Licinius'), ('killedr', 'http://www.wikidata.org/entity/Q8413'), ('killed', 'Constantine the Great')]
2


## Task 6
Find the BGP for war

In [56]:
queryString = """
SELECT DISTINCT ?namer ?name
WHERE {
    ?namer sc:name ?name.
    FILTER regex(?name, "^war$", "i")
}
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('namer', 'http://www.wikidata.org/entity/Q37732336'), ('name', 'WAR')]
[('namer', 'http://www.wikidata.org/entity/Q198'), ('name', 'war')]
[('namer', 'http://www.wikidata.org/entity/Q2058013'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q331811'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q2548927'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q7968362'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q1307369'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q4026034'), ('name', 'WAR')]
[('namer', 'http://www.wikidata.org/entity/Q2295871'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q640673'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q18891370'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q19053434'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q19115023'), ('name', 'War')]
[('namer', 'http://www.wikidata.org/entity/Q29727112'), ('

In [57]:
queryString = """
SELECT DISTINCT ?namer ?name
WHERE {
    ?namer sc:name ?name.
    FILTER regex(?name, "^war$", "")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('namer', 'http://www.wikidata.org/entity/Q198'), ('name', 'war')]
[('namer', 'http://www.wikidata.org/entity/Q29727112'), ('name', 'war')]
[('namer', 'http://www.wikidata.org/entity/Q26883397'), ('name', 'war')]
3


I have just 20 results, I may try them one by one. I noticed that most "general" concepts have a lower IRI number, so I started from the lowest, `wd:Q198`:

In [58]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q198 ?pr ?or .
    ?or sc:name ?o.
    ?pr sc:name ?p.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1151'), ('p', "topic's main Wikimedia portal"), ('or', 'http://www.wikidata.org/entity/Q10632410'), ('o', 'Portal:War')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q678259'), ('o', 'Nordisk familjebok')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q4114391'), ('o', 'Sytin Military Encyclopedia')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q106727050'), ('o', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q4086271'), ('o', 'Bible Encyclopedia of Archimandrite Nicephorus')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.w

As expected, it's the right one.

Final query for this task

In [59]:
queryString = """
SELECT DISTINCT ?namer ?name
WHERE {
    ?namer wdt:P2579 wd:Q192781.
    ?namer sc:name ?name.
    FILTER regex(?name, "^war$", "i")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('namer', 'http://www.wikidata.org/entity/Q198'), ('name', 'war')]
1


## Task 7
Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

In [60]:
queryString = """
SELECT DISTINCT ?warr ?war
WHERE {
    ?warr wdt:P31 wd:Q198. # instances of war
    ?warr sc:name ?war.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('warr', 'http://www.wikidata.org/entity/Q1362238'), ('war', 'World War I outside Europe')]
[('warr', 'http://www.wikidata.org/entity/Q1968718'), ('war', 'Great Syrian Revolt')]
[('warr', 'http://www.wikidata.org/entity/Q617070'), ('war', 'First Anglo-Mysore War')]
[('warr', 'http://www.wikidata.org/entity/Q20987787'), ('war', 'Esen Buqa–Ayurbarwada war')]
[('warr', 'http://www.wikidata.org/entity/Q3902588'), ('war', 'Little War in Hungary')]
[('warr', 'http://www.wikidata.org/entity/Q1640107'), ('war', 'Ifni War')]
[('warr', 'http://www.wikidata.org/entity/Q17512479'), ('war', 'Arab Winter')]
[('warr', 'http://www.wikidata.org/entity/Q22948744'), ('war', 'Rebellion of the Alpujarras')]
[('warr', 'http://www.wikidata.org/entity/Q1456086'), ('war', 'Nez Perce War')]
[('warr', 'http://www.wikidata.org/entity/Q2427419'), ('war', 'Guaraní War')]
[('warr', 'http://www.wikidata.org/entity/Q2471586'), ('war', 'Pomeranian War')]
[('warr', 'http://www.wikidata.org/entity/Q438252'), ('w

In [61]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q199955 ?pr ?or.
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 100 
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q326029'), ('o', 'Duchy of Brunswick')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q819125'), ('o', 'Province of Hanover')]
[('pr', 'http://www.wikidata.org/prop/direct/P2936'), ('p', 'language used'), ('or', 'http://www.wikidata.org/entity/Q150'), ('o', 'French')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q15864'), ('o', 'United Kingdom of the Netherlands')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q174193'), ('o', 'United Kingdom of Great Britain and Ireland')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q188586'), ('o', 'Kingdom of Sicily')]
[('pr', 'http://www.wikidata.org/prop/dir

Final query for this task

In [62]:
queryString = """
SELECT DISTINCT ?warr ?war
WHERE {
    ?warr wdt:P31  wd:Q198;  # a war
          wdt:P710 wd:Q2277. # partecipated by the roman empire
    
    ?warr sc:name ?war.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('warr', 'http://www.wikidata.org/entity/Q6254'), ('war', 'Kitos War')]
[('warr', 'http://www.wikidata.org/entity/Q500409'), ('war', 'Roman–Persian Wars')]
[('warr', 'http://www.wikidata.org/entity/Q202578'), ('war', 'First Jewish-Roman War')]
[('warr', 'http://www.wikidata.org/entity/Q1533828'), ('war', 'Roman–Parthian War of 58–63')]
[('warr', 'http://www.wikidata.org/entity/Q3651751'), ('war', "Trajan's Parthian campaign")]
[('warr', 'http://www.wikidata.org/entity/Q933963'), ('war', 'Marcomannic Wars')]
[('warr', 'http://www.wikidata.org/entity/Q3651748'), ('war', 'Parthian war of Caracalla')]
[('warr', 'http://www.wikidata.org/entity/Q1790268'), ('war', 'Roman–Parthian War of 161–166')]
[('warr', 'http://www.wikidata.org/entity/Q84850006'), ('war', 'Roman–Parthian War of 195–198')]
9


## Task 8
Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).

Here I did the other way around. Since I have no battles (and just a generic "battles" instance), I just picked up a random battle and saw how was the sorrounding schema.

In [63]:
queryString = """
SELECT DISTINCT ?battler ?battle
WHERE {
    ?battler wdt:P31 wd:Q178561. # instances of battles
    ?battler sc:name ?battle.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('battler', 'http://www.wikidata.org/entity/Q15728772'), ('battle', 'Battle of the Ligurian Sea')]
[('battler', 'http://www.wikidata.org/entity/Q15905209'), ('battle', 'Battle of Desna')]
[('battler', 'http://www.wikidata.org/entity/Q15918532'), ('battle', 'Action at Mechili, 24 January 1941')]
[('battler', 'http://www.wikidata.org/entity/Q15928518'), ('battle', 'battle of Qingshuping')]
[('battler', 'http://www.wikidata.org/entity/Q697621'), ('battle', 'Siege of Fort Zeelandia')]
[('battler', 'http://www.wikidata.org/entity/Q1497969'), ('battle', "Battle of Sailor's Creek")]
[('battler', 'http://www.wikidata.org/entity/Q194329'), ('battle', 'Battle of Five Forks')]
[('battler', 'http://www.wikidata.org/entity/Q4872730'), ('battle', 'Battle of White Oak Swamp')]
[('battler', 'http://www.wikidata.org/entity/Q913604'), ('battle', 'Battle of Alma')]
[('battler', 'http://www.wikidata.org/entity/Q16207132'), ('battle', 'Battle of Constantinople')]
[('battler', 'http://www.wikidata.

In [64]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q15728772 ?pr ?or.
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('p', 'part of'), ('or', 'http://www.wikidata.org/entity/Q702176'), ('o', 'Battle of the Mediterranean')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('p', 'instance of'), ('or', 'http://www.wikidata.org/entity/Q178561'), ('o', 'battle')]
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('p', 'location'), ('or', 'http://www.wikidata.org/entity/Q42820'), ('o', 'Ligurian Sea')]
3


In [65]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q702176 ?pr ?or.
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('p', 'location'), ('or', 'http://www.wikidata.org/entity/Q4918'), ('o', 'Mediterranean Sea')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('p', 'part of'), ('or', 'http://www.wikidata.org/entity/Q696817'), ('o', 'Mediterranean and Middle East theatre of World War II')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('p', 'instance of'), ('or', 'http://www.wikidata.org/entity/Q831663'), ('o', 'military campaign')]
[('pr', 'http://www.wikidata.org/prop/direct/P910'), ('p', "topic's main category"), ('or', 'http://www.wikidata.org/entity/Q8218839'), ('o', 'Category:Battle of the Mediterranean')]
4


Investigating the relationship between war and battle:

In [66]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q17026630 ?pr ?or.
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('p', 'location'), ('or', 'http://www.wikidata.org/entity/Q497801'), ('o', 'Tolosa')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q145'), ('o', 'United Kingdom')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('p', 'instance of'), ('or', 'http://www.wikidata.org/entity/Q178561'), ('o', 'battle')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('p', 'country'), ('or', 'http://www.wikidata.org/entity/Q29'), ('o', 'Spain')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('p', 'part of'), ('or', 'http://www.wikidata.org/entity/Q152499'), ('o', 'Peninsular War')]
[('pr', 'http://www.wikidata.org/prop/direct/P710'), ('p', 'participant'), ('or', 'http://www.wikidata.org/entity/Q71084'), ('o', 'First French Empire')]
6


We found that the battle is part the peninsular war, let's explore that entity to ensure is similar to the war we found in query 7:

In [67]:
queryString = """
SELECT DISTINCT ?pr ?p ?or ?o
WHERE {
    wd:Q152499 ?pr ?or.
    ?pr sc:name ?p.
    ?or sc:name ?o.
}
LIMIT 100 
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('p', 'instance of'), ('or', 'http://www.wikidata.org/entity/Q1006311'), ('o', 'war of national liberation')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('p', 'has part'), ('or', 'http://www.wikidata.org/entity/Q1190696'), ('o', 'War of the Oranges')]
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('p', 'location'), ('or', 'http://www.wikidata.org/entity/Q12837'), ('o', 'Iberian Peninsula')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q3181656'), ('o', 'The Nuttall Encyclopædia')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q4114391'), ('o', 'Sytin Military Encyclopedia')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('p', 'described by source'), ('or', 'http://www.wikidata.org/entity/Q602358'), ('o', 'Brockhaus and Efron Encyclopedic Dictionary')

Now I have everything I need to reconstruct the schema.


Final query for this task (without and with "OPTIONAL"):

In [68]:
queryString = """
SELECT ?warr (SAMPLE(?war) AS ?wars)
(SAMPLE(?opponentr) AS ?opponentrs)
(SAMPLE(?opponent) AS ?opponents)
(COUNT(?battler) as ?battles)
WHERE {
    ?warr wdt:P31  wd:Q198;    # a war
          wdt:P710 wd:Q2277;   # partecipated by the roman empire
          wdt:P710 ?opponentr. # and an opponent
          
    FILTER ( ?opponentr != wd:Q2277 )

         
    ?battler wdt:P31  wd:Q178561;
             wdt:P361 ?warr.
       
       
    ?warr sc:name ?war.
    ?opponentr sc:name ?opponent.
}
GROUP BY ?warr 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('warr', 'http://www.wikidata.org/entity/Q202578'), ('wars', 'First Jewish-Roman War'), ('opponentrs', 'http://www.wikidata.org/entity/Q56273996'), ('opponents', 'Judean Free Government'), ('battles', '4')]
[('warr', 'http://www.wikidata.org/entity/Q933963'), ('wars', 'Marcomannic Wars'), ('opponentrs', 'http://www.wikidata.org/entity/Q308885'), ('opponents', 'Quadi'), ('battles', '3')]
[('warr', 'http://www.wikidata.org/entity/Q500409'), ('wars', 'Roman–Persian Wars'), ('opponentrs', 'http://www.wikidata.org/entity/Q83891'), ('opponents', 'Sasanian Empire'), ('battles', '44')]
3


In [69]:
queryString = """
SELECT ?warr (SAMPLE(?war) AS ?wars)
(SAMPLE(?opponentr) AS ?opponentrs)
(SAMPLE(?opponent) AS ?opponents)
(COUNT(?battler) as ?battles)
WHERE {
    ?warr wdt:P31  wd:Q198;    # a war
          wdt:P710 wd:Q2277.   # partecipated by the roman empire
          
          
    OPTIONAL {
        ?warr wdt:P710 ?opponentr. # and an opponent
        FILTER ( ?opponentr != wd:Q2277 )
    }


    OPTIONAL {
    ?battler wdt:P31  wd:Q178561;
             wdt:P361 ?warr.    
    }

       
       
    ?warr sc:name ?war.
    ?opponentr sc:name ?opponent.
}
GROUP BY ?warr 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('warr', 'http://www.wikidata.org/entity/Q1790268'), ('wars', 'Roman–Parthian War of 161–166'), ('opponentrs', 'http://www.wikidata.org/entity/Q1986139'), ('opponents', 'Parthian Empire'), ('battles', '0')]
[('warr', 'http://www.wikidata.org/entity/Q3651748'), ('wars', 'Parthian war of Caracalla'), ('opponentrs', 'http://www.wikidata.org/entity/Q1986139'), ('opponents', 'Parthian Empire'), ('battles', '0')]
[('warr', 'http://www.wikidata.org/entity/Q1533828'), ('wars', 'Roman–Parthian War of 58–63'), ('opponentrs', 'http://www.wikidata.org/entity/Q208404'), ('opponents', 'Kingdom of Armenia'), ('battles', '0')]
[('warr', 'http://www.wikidata.org/entity/Q202578'), ('wars', 'First Jewish-Roman War'), ('opponentrs', 'http://www.wikidata.org/entity/Q56273996'), ('opponents', 'Judean Free Government'), ('battles', '4')]
[('warr', 'http://www.wikidata.org/entity/Q6254'), ('wars', 'Kitos War'), ('opponentrs', 'http://www.wikidata.org/entity/Q7325'), ('opponents', 'Jewish people'), ('